# MET API WALKTHROUGH
### MET X SDS Capstone
### A step by step tutorial on extracting images and metadata from the MET API

**Table of Contents:**

[Important Links + Information](#info)

[Installation](#okay)

* [Information](#legal)

* [Required Packages/Libraries](#ethics)

[Reading in The Metadata](#data)

* [Data Cleaning](#onepage)

* [Interacting with the MET API](#extract)

* [Putting extracted data in a dataframe](#dataframe)





## <a name="info"></a> Required Links + Information

Download the csv file that contains all the metadata(Artist Name, Medium, etc) associated with each object : 
https://github.com/metmuseum/openaccess/blob/master/MetObjects.csv

    
For more information about the Metropolitan Museum of Art Collection API, please visit the Open Access Github page https://github.com/metmuseum/openaccess.
    

**Access to the API** 

At this time, we do not require API users to register or obtain an API key to use the service.

## <a name="okay"></a> Installation

**Step 1 : First we load the modules and functions we will need: **

In [2]:
import requests
import json
from urllib import request
from urllib.request import urlopen
from urllib.request import HTTPError
import urllib
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import os

In [4]:
metdata = pd.read_csv('MetObjects.csv') 
# reading the csv file


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


How to calculate the size of the file? How to determine the number of object within a file?

Number of objects within the collection 

In [10]:
len(metdata)

471728

In [6]:
metdata.shape[0]

471728

In [7]:
metdata.columns

Index(['Object Number', 'Is Highlight', 'Is Public Domain', 'Object ID',
       'Department', 'Object Name', 'Title', 'Culture', 'Period', 'Dynasty',
       'Reign', 'Portfolio', 'Artist Role', 'Artist Prefix',
       'Artist Display Name', 'Artist Display Bio', 'Artist Suffix',
       'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date',
       'Artist End Date', 'Object Date', 'Object Begin Date',
       'Object End Date', 'Medium', 'Dimensions', 'Credit Line',
       'Geography Type', 'City', 'State', 'County', 'Country', 'Region',
       'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification',
       'Rights and Reproduction', 'Link Resource', 'Metadata Date',
       'Repository', 'Tags'],
      dtype='object')

In [13]:
metdata.head(5)

,Object Number,Is Highlight,Is Public Domain,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,...,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository,Tags
0,1979.486.1,False,False,1,The American Wing,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/1,NaN,"Metropolitan Museum of Art, New York, NY",NaN
1,1980.264.5,False,False,2,The American Wing,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,"Metropolitan Museum of Art, New York, NY",NaN
2,67.265.9,False,False,3,The American Wing,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,"Metropolitan Museum of Art, New York, NY",NaN
3,67.265.10,False,False,4,The American Wing,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,"Metropolitan Museum of Art, New York, NY",NaN
4,67.265.11,False,False,5,The American Wing,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,"Metropolitan Museum of Art, New York, NY",NaN
5,67.265.12,False,False,6,The American Wing,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/6,NaN,"Metropolitan Museum of Art, New York, NY",NaN
6,67.265.13,False,False,7,The American Wing,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/7,2019-02-01T10:50:49.477Z,"Metropolitan Museum of Art, New York, NY",Birds|Coins
7,67.265.14,False,False,8,The American Wing,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/8,2019-07-31T03:00:40.447Z,"Metropolitan Museum of Art, New York, NY",Eagles|Men|Profiles
8,67.265.15,False,False,9,The American Wing,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/9,NaN,"Metropolitan Museum of Art, New York, NY",NaN
9,1979.486.3,False,False,10,The American Wing,Coin,Two-and-a-half-dollar Indian Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/10,NaN,"Metropolitan Museum of Art, New York, NY",NaN


In [14]:
metdata.sample(5)

,Object Number,Is Highlight,Is Public Domain,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,...,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository,Tags
142982,74.51.4257,False,True,243669,Greek and Roman Art,Ring with carbuncle,Ring with carbuncle,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Gold and Silver,NaN,http://www.metmuseum.org/art/collection/search...,2019-02-01T10:50:49.477Z,"Metropolitan Museum of Art, New York, NY",Rings
311657,1975.1.2062,False,True,460939,Robert Lehman Collection,Chair,Papier-mâchè side chair,British (?),NaN,NaN,...,NaN,NaN,NaN,NaN,Woodwork-Furniture,NaN,http://www.metmuseum.org/art/collection/search...,NaN,"Metropolitan Museum of Art, New York, NY",NaN
410414,2005.100.588.1.51,False,False,704912,Photographs,Photograph; Carte-de-visite,Clara Pilvois dans le Diavolina,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Photographs,NaN,http://www.metmuseum.org/art/collection/search...,NaN,"Metropolitan Museum of Art, New York, NY",NaN
47026,JP1988,False,True,54690,Asian Art,Print,NaN,Japan,Edo period (1615–1868),NaN,...,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,2019-03-08T03:00:45.753Z,"Metropolitan Museum of Art, New York, NY",Netsuke|Men
403049,63.350.247.218.132,False,False,692687,Drawings and Prints,Print,"Albert G. Cutler, Billiards, from Mecca & Hass...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Prints|Ephemera,NaN,http://www.metmuseum.org/art/collection/search...,2019-02-01T10:50:49.477Z,"Metropolitan Museum of Art, New York, NY",Men|Portraits|Billiards
311592,1975.1.1361,False,True,460870,Robert Lehman Collection,Plaquette,The Last Supper,South Netherlandish or German,NaN,NaN,...,NaN,NaN,NaN,NaN,Plaquettes,NaN,http://www.metmuseum.org/art/collection/search...,2019-10-10T14:54:23.72Z,"Metropolitan Museum of Art, New York, NY",Last Supper|Christ
151359,26.31.444,False,False,252366,Greek and Roman Art,Kylix,Kylix,"Greek, Attic",Late Archaic,NaN,...,NaN,NaN,NaN,NaN,Vases,NaN,http://www.metmuseum.org/art/collection/search...,NaN,"Metropolitan Museum of Art, New York, NY",NaN
63008,2005.411.301,False,False,72877,Asian Art,Seal,劉一聞稚柳 巴林石|Zhiliu,China,NaN,NaN,...,NaN,NaN,NaN,NaN,Seals,NaN,http://www.metmuseum.org/art/collection/search...,2019-04-02T03:00:47.7Z,"Metropolitan Museum of Art, New York, NY",NaN
284907,63.350.202.60.13,False,True,422317,Drawings and Prints,Photograph,"Edwin Booth, from the Actresses and Celebritie...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Photographs|Ephemera,NaN,http://www.metmuseum.org/art/collection/search...,2019-02-01T10:50:49.477Z,"Metropolitan Museum of Art, New York, NY",Men|Portraits|Actors
54884,38.25.122,False,False,63321,Asian Art,Teabowl,NaN,Japan,NaN,NaN,...,NaN,NaN,NaN,NaN,Ceramics,NaN,http://www.metmuseum.org/art/collection/search...,NaN,"Metropolitan Museum of Art, New York, NY",NaN


In [15]:
metdata.tail(5)

,Object Number,Is Highlight,Is Public Domain,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,...,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository,Tags
471718,51.644.76(56),False,False,840964,Drawings and Prints,Photographs Ornament & Architecture,Harold Brown Esq.: Design for Sofa and Chair,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Drawings|Ornament & Architecture,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-17T04:42:18.47Z,"Metropolitan Museum of Art, New York, NY",NaN
471719,51.644.76(57),False,False,840965,Drawings and Prints,Photographs Ornament & Architecture,Harold Brown Esq.: Design for Chair,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Drawings|Ornament & Architecture,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-17T04:42:18.47Z,"Metropolitan Museum of Art, New York, NY",NaN
471720,17.3.756-1006,False,False,840969,Drawings and Prints,Print,Portrait of Cave Underhill,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-18T04:43:54.343Z,"Metropolitan Museum of Art, New York, NY",NaN
471721,24.63.1705,False,False,840972,Drawings and Prints,Print,Portrait of the Lady Abbess of the English Nun...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-18T04:43:54.343Z,"Metropolitan Museum of Art, New York, NY",NaN
471722,32.35(194),False,False,841019,Drawings and Prints,Print,Portrait of Lady Francis Byron,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-17T12:47:59.83Z,"Metropolitan Museum of Art, New York, NY",NaN
471723,80.3.517,False,False,841021,Drawings and Prints,drawing,Mediterranean Landscape with a Villa in the Di...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Drawings,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-17T12:47:59.83Z,"Metropolitan Museum of Art, New York, NY",NaN
471724,80.3.468,False,False,841024,Drawings and Prints,drawing,Portrait of a Man,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Drawings,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-18T04:43:54.343Z,"Metropolitan Museum of Art, New York, NY",NaN
471725,01.2.353,False,False,841025,Drawings and Prints,Print,"Portrait of Lisabetta du Parc, ""La Francesina""",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-18T04:41:00.3Z,"Metropolitan Museum of Art, New York, NY",NaN
471726,62.659.5,False,False,841032,Drawings and Prints,Print,18th century print with rhyme warning parents ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-18T04:41:00.3Z,"Metropolitan Museum of Art, New York, NY",NaN
471727,64.500.29,False,False,841033,Drawings and Prints,Print,Gathering Wheat,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,2020-01-18T04:41:00.3Z,"Metropolitan Museum of Art, New York, NY",NaN


In [16]:
metdata.describe()

,Object ID,Object Begin Date,Object End Date
count,471728.000000,471728.000000,471728.000000
mean,382220.365058,1290.890859,1392.391669
std,233572.895993,1723.402230,1137.411817
min,1.000000,-400000.000000,-240000.000000
25%,208315.750000,1525.000000,1580.000000
50%,368553.500000,1800.000000,1835.000000
75%,559418.250000,1890.000000,1903.000000
max,841033.000000,5000.000000,15335.000000


In [17]:
metdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471728 entries, 0 to 471727
Data columns (total 44 columns):
Object Number              471728 non-null object
Is Highlight               471728 non-null bool
Is Public Domain           471728 non-null bool
Object ID                  471728 non-null int64
Department                 471728 non-null object
Object Name                469909 non-null object
Title                      441652 non-null object
Culture                    206051 non-null object
Period                     88740 non-null object
Dynasty                    23137 non-null object
Reign                      11159 non-null object
Portfolio                  23044 non-null object
Artist Role                267759 non-null object
Artist Prefix              88815 non-null object
Artist Display Name        269853 non-null object
Artist Display Bio         230705 non-null object
Artist Suffix              11969 non-null object
Artist Alpha Sort          269819 non-null object


# Data Cleaning 

In [10]:
metdata = metdata.drop_duplicates()

# API 

In [4]:
urlid = 'https://collectionapi.metmuseum.org/public/collection/v1/objects'
objectID = requests.get(urlid).json()
content = json.dumps(objectID, sort_keys=True)

In [5]:
IDdf = pd.DataFrame(objectID["objectIDs"])
IDdf.columns=['objectIDs']
IDdf.head()

,objectIDs
0,1
1,2
2,3
3,4
4,5


In [6]:
IDdf = IDdf.sort_values(by='objectIDs',ascending=True).reset_index()
IDdf.head() 

,index,objectIDs
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5


In [7]:
len(IDdf)

471785

this number will equal the size of the csv file 

In [ ]:
ct= 0
empty = []  
for i in range(0, len(IDdf)):
    url = 'https://collectionapi.metmuseum.org/public/collection/v1/objects/{}'.format(IDdf['objectIDs'][i])    
    objectinfo = requests.get(url).json()    
    imageurl = objectinfo['primaryImage']
    table = [IDdf['objectIDs'][i], imageurl]
    empty.append(table)
    ct+=1
    if ct % 100==0: #  see the progress for every 200 data 
        current = pd.DataFrame(empty,columns=['objectID','URL'])
        current.to_csv('MetIDs.csv',index = None, header=True)
        print (ct)

100
200
300
400
500
600


In [9]:
objectID = requests.get("https://collectionapi.metmuseum.org/public/collection/v1/objects")


In [10]:
print(objectID.status_code)

200


if you receive a ‘200’ code - that tells us that our request was successful



if you receive a ‘403’ code - that tells us that our request was unsuccessful

In [11]:
url = "https://collectionapi.metmuseum.org/public/collection/v1/objects"
JSONContent = requests.get(url).json()
#JSON: JavaScript Object Notation.
#JSON is a syntax for storing and exchanging data.

ids = JSONContent['objectIDs']

In [16]:
for id_ in IDdf['objectIDs']:
    content = requests.get("https://collectionapi.metmuseum.org/public/collection/v1/objects/" + str(id_)).json()
    #print(content)
    url = content['primaryImage']
    if url=='':
        continue
    print(url)
    break

https://images.metmuseum.org/CRDImages/ad/original/204788.jpg
